In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from collections import Counter

In [2]:
rate = pd.read_csv('ratings.csv', parse_dates = ['Date'])
rem = pd.read_csv('remarks.csv', parse_dates = ['remarkDate'])
rem_sup = pd.read_csv('remarks_supp_opp.csv')
train = pd.read_csv('train.csv', parse_dates = ['lastratingdate'])
test = pd.read_csv('test.csv',parse_dates = ['lastratingdate'])

/opt/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def extract_dates(df,column):
    df[column+'_yy'] = df[column].apply(lambda x: x.year)
    df[column+'_mm'] = df[column].apply(lambda x: x.month)
    df[column+'_dd'] = df[column].apply(lambda x: x.day)
    df = df.drop(columns=column)
    return df

In [4]:
def score_function(y_pred,y):
    y_pred = np.asarray(y_pred)
    y = np.asarray(y)
    assert y.shape[0] == y_pred.shape[0]
    n = y.shape[0]
    total = 4*np.sum(y) + n
    score = 0
    for i in range(n):
        if y_pred[i] == y[i]:
            if y[i] == 1:
                score += 5
            elif y[i] == 0:
                score += 1
    return score/total, score, total

In [5]:
rate = rate.loc[rate['emp'] >= 0]
rem = rem.loc[rem['emp'] >= 0]
rem_sup = rem_sup.loc[rem_sup['emp'] >= 0]

rem = rem.dropna()
rem_sup = rem_sup.dropna()
rate = rate.dropna()

rem = rem.drop_duplicates()
rem_sup = rem_sup.drop_duplicates()
rate = rate.drop_duplicates()

In [6]:
rate = extract_dates(rate,'Date')
rem = extract_dates(rem,'remarkDate')
train = extract_dates(train,'lastratingdate')
test = extract_dates(test,'lastratingdate')

In [7]:
rem_sup = rem_sup.drop(columns='oppose')

In [8]:
rem['txt_len'] = rem['txt'].apply(lambda x: len(x))
rem = rem.drop(columns='txt')

In [9]:
rate

,emp,comp,rating,Date_yy,Date_mm,Date_dd
0,31,bnivzbfi,4,2016,1,2
1,33,bnivzbfi,4,2016,1,2
2,79,bnivzbfi,4,2016,1,2
3,94,bnivzbfi,4,2016,1,2
4,16,bnivzbfi,2,2016,1,2
...,...,...,...,...,...,...
221227,17,zptfoxyq,3,2016,3,6
221228,17,zptfoxyq,4,2016,7,6
221229,17,zptfoxyq,4,2016,6,13
221230,17,zptfoxyq,4,2016,6,28


In [10]:
rate_list = []
emps = rate['emp'].unique()
comps = rate['comp'].unique()
for comp in tqdm(comps):
    comp_wise_list = rate.loc[rate['comp'] == comp]
    for emp in emps:
        r = comp_wise_list.loc[rate['emp'] == emp]
        if r.shape[0]:
            l = list(r['rating'])
            list_of_rate = np.zeros(4,dtype=np.int32)
            count_dict = Counter(l)
            for i in range(1,5):
                list_of_rate[i-1] = count_dict[i]
            rate_list.append([emp,comp,list_of_rate,r['rating'].shape[0],\
                              r.iloc[-1]['Date_yy'],r.iloc[-1]['Date_mm'],r.iloc[-1]['Date_dd']])

100%|██████████| 37/37 [00:43<00:00,  1.55s/it]


In [11]:
rate_df = pd.DataFrame(rate_list,columns=['emp','comp','rating','freq',\
                                          'lastratingdate_yy','lastratingdate_mm','lastratingdate_dd'])
rate_df

,emp,comp,rating,freq,lastratingdate_yy,lastratingdate_mm,lastratingdate_dd
0,31,bnivzbfi,"[0, 0, 0, 53]",53,2017,3,19
1,33,bnivzbfi,"[0, 0, 1, 252]",253,2017,3,20
2,79,bnivzbfi,"[0, 0, 0, 194]",194,2017,3,20
3,94,bnivzbfi,"[25, 22, 20, 33]",100,2017,10,3
4,16,bnivzbfi,"[13, 6, 2, 69]",90,2017,2,16
...,...,...,...,...,...,...,...
4372,854,wsmblohy,"[0, 0, 3, 0]",3,2017,3,20
4373,855,wsmblohy,"[0, 0, 1, 0]",1,2017,3,17
4374,858,wsmblohy,"[0, 0, 1, 0]",1,2017,3,17
4375,17,zptfoxyq,"[0, 0, 3, 4]",7,2016,6,28


In [12]:
train_rate_list = []
train_rate_freq = []
# a,b,c = [],[],[]
for _,row in train.iterrows():
    r = rate_df[(rate_df['emp'] == row['emp']) & (rate_df['comp'] == row['comp'])]
    train_rate_list.extend(list(r['rating']))
    train_rate_freq.extend(list(r['freq']))
#     a.(list(r['lastratingdate_yy']))
#     b.append(list(r['lastratingdate_mm']))
#     c.append(list(r['lastratingdate_dd']))

In [13]:
train['ratings'] = train_rate_list
train['freq'] = train_rate_freq
rate_array = np.asarray(list(train['ratings']))
for i in range(rate_array.shape[1]):
    train['rate_'+str(i+1)] = rate_array[:,i]
train = train.drop(columns='ratings')
train

,id,emp,comp,left,lastratingdate_yy,lastratingdate_mm,lastratingdate_dd,freq,rate_1,rate_2,rate_3,rate_4
0,2228,939,phcvroct,1,2016,10,17,1,0,0,1,0
1,4349,250,jblrepyr,0,2017,3,19,112,0,4,18,90
2,945,134,ewpvmfbc,0,2016,9,21,26,0,2,24,0
3,4553,164,wsmblohy,0,2017,3,17,17,2,7,7,1
4,941,129,ewpvmfbc,0,2016,4,4,5,0,1,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3521,783,87,ocsicwng,0,2017,3,20,71,1,6,63,1
3522,3655,14,jnvpfmup,0,2017,3,17,12,0,0,11,1
3523,1460,53,xccmgbjz,0,2017,9,3,2,0,0,2,0
3524,842,168,ocsicwng,0,2017,1,3,12,0,0,10,2


In [14]:
# l = train[train['left'] == 1]
# nl = train[train['left'] == 0]
# a = np.asarray(list(l['ratings']))
# b = np.asarray(list(nl['ratings']))
# lf = np.argmax(a,axis=1)
# print(Counter(lf))
# nlf = np.argmax(b,axis=1)
# print(Counter(nlf))

# print(np.sum(a,axis=0))
# print(np.sum(b,axis=0))

People who left office have rated lower on average (1-2) is more than the people who stayed. They have rated more (3-4). The median and mean of frequency of people who didn't leave the office is more than the people who left the office.

In [15]:
test_rate_list = []
test_rate_freq = []
for _,row in test.iterrows():
    r = rate_df[(rate_df['emp'] == row['emp']) & (rate_df['comp'] == row['comp'])]
    test_rate_list.extend(list(r['rating']))
    test_rate_freq.extend(list(r['freq']))

In [16]:
test['ratings'] = test_rate_list
test['freq'] = test_rate_freq
rate_array = np.asarray(list(test['ratings']))
for i in range(rate_array.shape[1]):
    test['rate_'+str(i+1)] = rate_array[:,i]
test = test.drop(columns='ratings')
test

,id,emp,comp,lastratingdate_yy,lastratingdate_mm,lastratingdate_dd,freq,rate_1,rate_2,rate_3,rate_4
0,353,509,bnivzbfi,2017,3,20,43,0,0,0,43
1,732,22,ocsicwng,2017,3,17,69,4,52,13,0
2,3076,2,siexkzzo,2017,9,1,18,0,10,8,0
3,2199,885,phcvroct,2016,1,10,83,2,18,55,8
4,2145,756,phcvroct,2017,6,2,9,4,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...
877,4022,36,ydqdpmvi,2016,5,10,74,2,21,51,0
878,3397,30,fqsozvpv,2017,3,14,2,0,0,0,2
879,1028,15,rujnkvse,2017,1,30,4,0,0,3,1
880,3179,127,siexkzzo,2017,3,17,27,0,7,18,2


In [17]:
rem

,emp,comp,remarkId,remarkDate_yy,remarkDate_mm,remarkDate_dd,txt_len
0,307,bnivzbfi,fvwadfrj,2017,3,20,62
1,382,bnivzbfi,xrrfkgap,2017,3,20,29
2,172,bnivzbfi,xkcrccwi,2017,3,20,27
3,135,bnivzbfi,lohhvtmo,2017,3,20,27
4,225,bnivzbfi,gpxxmoab,2017,3,20,33
...,...,...,...,...,...,...,...
81632,6,wsmblohy,lmkjqqgo,2017,2,18,81
81633,5,wsmblohy,pbxqvyto,2017,2,18,111
81634,10,wsmblohy,mngsaput,2017,2,17,126
81635,2,wsmblohy,eovaeyph,2017,2,17,53


Ignoring Remark dates for the time being.

In [18]:
remark_len = []
remark_freq = []
support_list = []
total_list = []
for _,row in tqdm(train.iterrows()):
    r = rem[(rem['emp'] == row['emp']) & (rem['comp'] == row['comp'])]
    if r.shape[0]:
        remark_len.append(np.mean(np.asarray(list(r['txt_len']))))
        remarks = r.remarkId.unique()
        support, total = 0, 0
        for remark in remarks:
            rel_list = rem_sup[rem_sup['remarkId'] == remark]
            support += Counter(rel_list['support'])[True]
            total += rel_list.shape[0]
        total_list.append(total)
        support_list.append(support)
    else:
        remark_len.append(0)
        total_list.append(0)
        support_list.append(0)
    remark_freq.append(r.shape[0])   

3526it [07:18,  8.04it/s]


In [19]:
train['max_rem_len'] = remark_len
train['rem_freq'] = remark_freq
train['sup'] = support_list
train['total'] = total_list
train

,id,emp,comp,left,lastratingdate_yy,lastratingdate_mm,lastratingdate_dd,freq,rate_1,rate_2,rate_3,rate_4,max_rem_len,rem_freq,sup,total
0,2228,939,phcvroct,1,2016,10,17,1,0,0,1,0,0.000000,0,0,0
1,4349,250,jblrepyr,0,2017,3,19,112,0,4,18,90,136.326923,52,385,420
2,945,134,ewpvmfbc,0,2016,9,21,26,0,2,24,0,45.800000,5,23,25
3,4553,164,wsmblohy,0,2017,3,17,17,2,7,7,1,21.200000,5,22,59
4,941,129,ewpvmfbc,0,2016,4,4,5,0,1,4,0,75.000000,2,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3521,783,87,ocsicwng,0,2017,3,20,71,1,6,63,1,74.444444,9,42,63
3522,3655,14,jnvpfmup,0,2017,3,17,12,0,0,11,1,28.500000,2,5,5
3523,1460,53,xccmgbjz,0,2017,9,3,2,0,0,2,0,0.000000,0,0,0
3524,842,168,ocsicwng,0,2017,1,3,12,0,0,10,2,82.333333,3,12,13


In [20]:
remark_len = []
remark_freq = []
support_list = []
total_list = []
for _,row in tqdm(test.iterrows()):
    r = rem[(rem['emp'] == row['emp']) & (rem['comp'] == row['comp'])]
    if r.shape[0]:
        remark_len.append(np.mean(np.asarray(list(r['txt_len']))))
        remarks = r.remarkId.unique()
        support, total = 0, 0
        for remark in remarks:
            rel_list = rem_sup[rem_sup['remarkId'] == remark]
            support += Counter(rel_list['support'])[True]
            total += rel_list.shape[0]
        total_list.append(total)
        support_list.append(support)
    else:
        remark_len.append(0)
        total_list.append(0)
        support_list.append(0)
    remark_freq.append(r.shape[0])   

882it [01:57,  3.78it/s]


Now, we are using maximum reamrk length, if we want we can use, average or some other order statistics.

In [21]:
test['max_rem_len'] = remark_len
test['rem_freq'] = remark_freq
test['sup'] = support_list
test['total'] = total_list
test

,id,emp,comp,lastratingdate_yy,lastratingdate_mm,lastratingdate_dd,freq,rate_1,rate_2,rate_3,rate_4,max_rem_len,rem_freq,sup,total
0,353,509,bnivzbfi,2017,3,20,43,0,0,0,43,86.000000,3,42,43
1,732,22,ocsicwng,2017,3,17,69,4,52,13,0,43.800000,5,26,27
2,3076,2,siexkzzo,2017,9,1,18,0,10,8,0,0.000000,0,0,0
3,2199,885,phcvroct,2016,1,10,83,2,18,55,8,67.500000,2,7,11
4,2145,756,phcvroct,2017,6,2,9,4,1,3,1,78.000000,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,4022,36,ydqdpmvi,2016,5,10,74,2,21,51,0,26.000000,1,11,11
878,3397,30,fqsozvpv,2017,3,14,2,0,0,0,2,72.000000,2,2,3
879,1028,15,rujnkvse,2017,1,30,4,0,0,3,1,0.000000,0,0,0
880,3179,127,siexkzzo,2017,3,17,27,0,7,18,2,0.000000,0,0,0


In [22]:
test # Check how are the train and test right here

,id,emp,comp,lastratingdate_yy,lastratingdate_mm,lastratingdate_dd,freq,rate_1,rate_2,rate_3,rate_4,max_rem_len,rem_freq,sup,total
0,353,509,bnivzbfi,2017,3,20,43,0,0,0,43,86.000000,3,42,43
1,732,22,ocsicwng,2017,3,17,69,4,52,13,0,43.800000,5,26,27
2,3076,2,siexkzzo,2017,9,1,18,0,10,8,0,0.000000,0,0,0
3,2199,885,phcvroct,2016,1,10,83,2,18,55,8,67.500000,2,7,11
4,2145,756,phcvroct,2017,6,2,9,4,1,3,1,78.000000,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,4022,36,ydqdpmvi,2016,5,10,74,2,21,51,0,26.000000,1,11,11
878,3397,30,fqsozvpv,2017,3,14,2,0,0,0,2,72.000000,2,2,3
879,1028,15,rujnkvse,2017,1,30,4,0,0,3,1,0.000000,0,0,0
880,3179,127,siexkzzo,2017,3,17,27,0,7,18,2,0.000000,0,0,0


In [23]:
y = train['left']
train1 = train.drop(columns='left')
train1

,id,emp,comp,lastratingdate_yy,lastratingdate_mm,lastratingdate_dd,freq,rate_1,rate_2,rate_3,rate_4,max_rem_len,rem_freq,sup,total
0,2228,939,phcvroct,2016,10,17,1,0,0,1,0,0.000000,0,0,0
1,4349,250,jblrepyr,2017,3,19,112,0,4,18,90,136.326923,52,385,420
2,945,134,ewpvmfbc,2016,9,21,26,0,2,24,0,45.800000,5,23,25
3,4553,164,wsmblohy,2017,3,17,17,2,7,7,1,21.200000,5,22,59
4,941,129,ewpvmfbc,2016,4,4,5,0,1,4,0,75.000000,2,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3521,783,87,ocsicwng,2017,3,20,71,1,6,63,1,74.444444,9,42,63
3522,3655,14,jnvpfmup,2017,3,17,12,0,0,11,1,28.500000,2,5,5
3523,1460,53,xccmgbjz,2017,9,3,2,0,0,2,0,0.000000,0,0,0
3524,842,168,ocsicwng,2017,1,3,12,0,0,10,2,82.333333,3,12,13


In [24]:
mergedata = train1.append(test)
testcount = len(test)
count = len(mergedata)-testcount
X_cat = mergedata.copy()

In [25]:
X_cat1 = pd.get_dummies(X_cat, columns=["comp"])
X_cat1

,id,emp,lastratingdate_yy,lastratingdate_mm,lastratingdate_dd,freq,rate_1,rate_2,rate_3,rate_4,...,comp_ugldwwzf,comp_ujplihug,comp_vcqsbirc,comp_vwcdylha,comp_wsmblohy,comp_xccmgbjz,comp_ydqdpmvi,comp_ylpksopb,comp_yodaczsb,comp_zptfoxyq
0,2228,939,2016,10,17,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,4349,250,2017,3,19,112,0,4,18,90,...,0,0,0,0,0,0,0,0,0,0
2,945,134,2016,9,21,26,0,2,24,0,...,0,0,0,0,0,0,0,0,0,0
3,4553,164,2017,3,17,17,2,7,7,1,...,0,0,0,0,1,0,0,0,0,0
4,941,129,2016,4,4,5,0,1,4,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,4022,36,2016,5,10,74,2,21,51,0,...,0,0,0,0,0,0,1,0,0,0
878,3397,30,2017,3,14,2,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
879,1028,15,2017,1,30,4,0,0,3,1,...,0,0,0,0,0,0,0,0,0,0
880,3179,127,2017,3,17,27,0,7,18,2,...,0,0,0,0,0,0,0,0,0,0


In [26]:
X = X_cat1[:count]
X_test = X_cat1[count:]
# X = X.loc[:,'emp':]
# X_test = X_test.loc[:,'emp':]
X

,id,emp,lastratingdate_yy,lastratingdate_mm,lastratingdate_dd,freq,rate_1,rate_2,rate_3,rate_4,...,comp_ugldwwzf,comp_ujplihug,comp_vcqsbirc,comp_vwcdylha,comp_wsmblohy,comp_xccmgbjz,comp_ydqdpmvi,comp_ylpksopb,comp_yodaczsb,comp_zptfoxyq
0,2228,939,2016,10,17,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,4349,250,2017,3,19,112,0,4,18,90,...,0,0,0,0,0,0,0,0,0,0
2,945,134,2016,9,21,26,0,2,24,0,...,0,0,0,0,0,0,0,0,0,0
3,4553,164,2017,3,17,17,2,7,7,1,...,0,0,0,0,1,0,0,0,0,0
4,941,129,2016,4,4,5,0,1,4,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3521,783,87,2017,3,20,71,1,6,63,1,...,0,0,0,0,0,0,0,0,0,0
3522,3655,14,2017,3,17,12,0,0,11,1,...,0,0,0,0,0,0,0,0,0,0
3523,1460,53,2017,9,3,2,0,0,2,0,...,0,0,0,0,0,1,0,0,0,0
3524,842,168,2017,1,3,12,0,0,10,2,...,0,0,0,0,0,0,0,0,0,0


In [27]:
X_test

,id,emp,lastratingdate_yy,lastratingdate_mm,lastratingdate_dd,freq,rate_1,rate_2,rate_3,rate_4,...,comp_ugldwwzf,comp_ujplihug,comp_vcqsbirc,comp_vwcdylha,comp_wsmblohy,comp_xccmgbjz,comp_ydqdpmvi,comp_ylpksopb,comp_yodaczsb,comp_zptfoxyq
0,353,509,2017,3,20,43,0,0,0,43,...,0,0,0,0,0,0,0,0,0,0
1,732,22,2017,3,17,69,4,52,13,0,...,0,0,0,0,0,0,0,0,0,0
2,3076,2,2017,9,1,18,0,10,8,0,...,0,0,0,0,0,0,0,0,0,0
3,2199,885,2016,1,10,83,2,18,55,8,...,0,0,0,0,0,0,0,0,0,0
4,2145,756,2017,6,2,9,4,1,3,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,4022,36,2016,5,10,74,2,21,51,0,...,0,0,0,0,0,0,1,0,0,0
878,3397,30,2017,3,14,2,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
879,1028,15,2017,1,30,4,0,0,3,1,...,0,0,0,0,0,0,0,0,0,0
880,3179,127,2017,3,17,27,0,7,18,2,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# train = pd.get_dummies(train, columns=["comp"])# merge with main df bridge_df on key values
# # train1 = train1.join(dum_df)
# train

In [29]:
# train_col = train.columns.tolist()
# train_col = [train_col[1]] + train_col[3:] + [train_col[2]]
# train = train[train_col]
# train

In [30]:
# req_cols = train_col[:-1]
# X = train.loc[:,req_cols]
# y = train['left']
# X

In [31]:
# test = pd.get_dummies(test, columns=["comp"])
# test

In [32]:
# test_col = test.columns.tolist()
# req_cols = [test_col[1]] + test_col[3:]
# req_cols
# X_test = test.loc[:,req_cols]
# X_test

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [43]:
import xgboost as xgb

In [48]:
# X = StandardScaler().fit_transform(X)
iters = 50
y_pred_list = np.zeros((X_test.shape[0],iters))
for i in tqdm(range(iters)):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.2)
    clf =  xgb.XGBClassifier()
    clf.fit(X_train, y_train)
#     print(score_function(clf.predict(X_train),y_train))
#     print(score_function(clf.predict(X_val),y_val))
    y_pred_list[:,i] = clf.predict(X_test)
#     print(score_function(np.ones(y_val.shape[0]),y_val))
#     print(score_function(np.zeros(y_val.shape[0]),y_val))    

100%|██████████| 50/50 [00:22<00:00,  2.45it/s]


In [49]:
y_pred = np.asarray(np.sum(y_pred_list,axis=1) > 0,dtype=np.int32)
np.sum(y_pred)

195

In [50]:
final_preds = pd.DataFrame({'id':list(test['id']),'left':y_pred})
final_preds

,id,left
0,353,0
1,732,0
2,3076,0
3,2199,1
4,2145,0
...,...,...
877,4022,0
878,3397,0
879,1028,0
880,3179,0


In [51]:
final_preds.to_csv('preds.csv',index=False)